# Import libraries and data set

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set()

df_NACC = pd.read_csv('./investigator_nacc46.csv')

C:\Users\Debraj\Anaconda3\envs\NLP_virtual\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (19,21,23,25,27,40,43,45,47,50,60,62,64,66,68,70,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,133,164,175,178,216,219,221,223,225,227,229,231,233,235,237,239,241,243,245,249,251,253,255,257,259,261,263,265,267,269,271,381,396,398,400,420,422,431,444,453,493,566,574,599,635,651,668,671,676,688,699,705,707,774,785,787,789,791,797,858,913,914,915,923,924,925,926,936,958,961,964) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_NACC.groupby(['NACCID','DEMENTED']).size()

# Check if there is any NULL value

In [ ]:
df_NACC['DEMENTED'].isnull().sum()
df_NACC.isnull().sum()

# Filtering the records of first visits

In [6]:

df_intial= df_NACC[df_NACC['PACKET']== "I"]
df_intial['DEMENTED'].isnull().sum()

0

# Checking the distribution of positive and negative examples in the data set

In [7]:
df_intial.groupby(['DEMENTED']).size()

DEMENTED
0    26478
1    14380
dtype: int64

In [8]:
#size of the intial visit record 
df_intial.shape

(40858, 965)

# Listing the columns to consider

In [9]:
col2use =['NACCID', 'SEX', 'HISPANIC', 'PRIMLANG', 'EDUC', 'MARISTAT', 'NACCLIVS', 'INDEPEND', \
       'RESIDENC', 'NACCAGEB', 'NACCFAM','DEMENTED' ]
col2test = ['SEX', 'HISPANIC', 'PRIMLANG', 'EDUC', 'MARISTAT', 'NACCLIVS', 'INDEPEND', \
         'RESIDENC', 'NACCAGEB', 'NACCFAM' ]

# Data cleaning continued, removing irrelevant records using the dictionary from the documentation of NACC in the Neuropathology (NP) Data Set (https://www.alz.washington.edu/NONMEMBER/NP/rdd_np.pdf) and NACC Uniform Data set (UDS) (https://www.alz.washington.edu/WEB/rdd_uds.pdf) dictionaries)

In [10]:
df_intial =df_intial[(df_intial['HISPANIC'] !=9)]
df_intial = df_intial[(df_intial['PRIMLANG'] !=8) & (df_intial['PRIMLANG'] !=9)]
df_intial = df_intial[(df_intial['EDUC'] <=36) & (df_intial['EDUC'] >=0)]
df_intial = df_intial[(df_intial['MARISTAT'] != 9)]
df_intial = df_intial[(df_intial['NACCLIVS'] != 9)]
df_intial = df_intial[(df_intial['INDEPEND'] != 9)]
df_intial = df_intial[(df_intial['RESIDENC'] != 9)] 
df_intial = df_intial[(df_intial['NACCAGEB'] >40)] 
df_intial = df_intial[(df_intial['NACCFAM'] !=-4) & (df_intial['NACCFAM'] != 9)]


In [11]:
# creating the cleaned data frame
df = df_intial[col2use]
df.reset_index(inplace= True)

# Importing sklearn libraries for classification

In [12]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import cross_val_score

# Forming the feature set and the corrosponding levels

In [13]:
Y = df['DEMENTED'].values
X = df[col2test].values

# Run different classifier models with Gridsearch to find the optimal set of parameters

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
import numpy as np
np.random.seed()
import random
random.seed()
random_state = np.random.RandomState(42)
from sklearn.ensemble import RandomForestClassifier
#Choose from any scaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
import random
random_state = np.random.RandomState(42)
#X_train, X_test, y_train, y_test = train_test_split(X_features, Y, test_size = 0.20)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state= random_state)
#Scale the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
# apply same transformation to test data
X_test = scaler.transform(X_test)
from sklearn import metrics
# number of trees
n_estimators = range(200,1000,200)
# maximum number of features to use at each split
max_features = ['auto','sqrt']
# maximum depth of the tree
max_depth = range(1,10,1)
# minimum number of samples to split a node
min_samples_split = range(2,10,2)
# criterion for evaluating a split
criterion = ['gini','entropy']

param_grid = {
    'bootstrap': [True],
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': min_samples_split,
    'n_estimators': n_estimators,
    'criterion' : criterion  
}

rf_model=RandomForestClassifier()

grid_rf = GridSearchCV(rf_model, param_grid = param_grid,
                          cv =5 , n_jobs = -1, verbose= 2)

# Fit the grid search to the data
grid_rf.fit(X_train, y_train)
print("the params\n", grid_rf.best_params_)
print("Best score: %0.4f" % grid_rf.best_score_)
final_model = grid_rf.best_estimator_

Y_pred = final_model.predict(X_test)

from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,Y_pred))
print("\n")
print(classification_report(y_test,Y_pred))

print("Training set score for RF: %f" % final_model.score(X_train , y_train))
print("Testing  set score for RF: %f" % final_model.score(X_test  , y_test ))
print("Accuracy:",metrics.accuracy_score(y_test, Y_pred))

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 1961 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done 3257 tasks      | elapsed: 19.8min
